# Notebook imports


In [1]:
import numpy as np
import pandas as pd

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/SpamData/03_Testing/test-target.txt'


VOCAB_SIZE = 2500

# Read and load features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[0:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2035,    0,    2],
       [5795, 2040,    0,    1],
       [5795, 2272,    0,    1],
       [5795, 2342,    0,    1],
       [5795, 2370,    0,    1]])

In [7]:
print('No of rows in training data', sparse_train_data.shape[0]) 
print('No of rows in testing data', sparse_test_data.shape[0]) 

No of rows in training data 258360
No of rows in testing data 117776


In [8]:
print("No of email in training data", np.unique(sparse_train_data[:,0]).size)

No of email in training data 4013


In [9]:
print("No of email in testing data", np.unique(sparse_test_data[:,0]).size)

No of email in testing data 1724


# Creating an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns= column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head

<bound method NDFrame.head of       DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
0          0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1          0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2          0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3          0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4          0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
...      ...       ... .. .. .. .. .. .. .. ..  ...   ...   ...   ...   ...   
5789       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5790       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5791       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5794       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5795       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

      2494  2495  249

# Creating a Full Matrix from a Sparse Matrix

In [14]:
def make_full_matrix(sparse_matrix, no_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Forms a Full matrix from a sparse matrix. Returns a pandas Dataframe.
    KeyWord Arguments:
    sparse_matrix -- numpy Array
    no_words --  size of the vocab. Total no of token
    doc_odx -- position of the document id in the sparse matrix, Default : 1st column
    word_idx -- position of the word id in the sparse matrix , Default: 2nd coulmn
    cat_idx -- position of the label (spam 1, ham 0), Default: 3rd column
    freq_idx -- position of occurence of the word in the sparse martrix, Default: 4th Column
    
    
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:,0])
    
    full_matrix = pd.DataFrame(index=index_names, columns= column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_no = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_no, 'DOC_ID'] = doc_no 
        full_matrix.at[doc_no, 'CATEGORY'] = label
        full_matrix.at[doc_no, word_id] = occurrence
        
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [15]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: total: 33.1 s
Wall time: 34.1 s


In [16]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the probability of Spam

In [18]:
full_train_data.CATEGORY.size

4013

In [19]:
full_train_data.CATEGORY.sum()

1248

In [20]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
prob_spam

0.310989284824321

## Total no of words or tokens

In [21]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [23]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [24]:
total_wc = email_lengths.sum()
total_wc

429241

# No of Tokens in spam and Ham Emails

In [25]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [26]:
spam_wc = spam_lengths.sum()
spam_wc

176334

In [27]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [28]:
ham_wc = ham_lengths.sum()
ham_wc

252907

In [29]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252907

In [30]:
spam_wc + nonspam_wc - total_wc

0

In [31]:
# Calcualting the average length of spam and non spam emails
print('Average no of words in spam emails {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average no of words in spam emails {:.0f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average no of words in spam emails 141
Average no of words in spam emails 91


# Summing the tokens occuring in spam

In [32]:
full_train_features.shape


(4013, 2500)

In [33]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_spam_tokens.shape

(1248, 2500)

In [36]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [37]:
summed_spam_tokens.tail()

2495     3
2496     2
2497    17
2498    15
2499     2
dtype: int64

## Summing the tokens Occuring in Ham

In [38]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [39]:
summed_ham_tokens.shape

(2500,)

In [40]:
summed_ham_tokens.tail()

2495    27
2496    35
2497    20
2498    15
2499    28
dtype: int64

## P(token | Spam) -  Probability that a token Occurs given the Email  is Spam

In [41]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
dtype: float64

In [42]:
prob_tokens_spam.sum()

1.0

 ## P(token | Ham) -  Probability that a token Occurs given the Email is NonSpam

In [43]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(token) - Probability that the token occurs

In [44]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [45]:
prob_tokens_all.sum()

1.0

# Saved the trained Model

In [46]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [47]:
sparse_test_data.shape

(117776, 4)

In [48]:
%%time
full_test_data = make_full_matrix(sparse_test_data, no_words=VOCAB_SIZE)

CPU times: total: 6min 58s
Wall time: 7min 7s


In [49]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [50]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)